In [1]:
# Add the parent directory of the current working directory to the Python path at runtime. 
# In order to import modules from the src directory.
import os
import sys 

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

In [20]:
import glob
import pickle

from src.stat_utils import *

results_dir = os.path.join(parent_dir, 'results/statistics')
full_model_file = os.path.join(results_dir, 'gee_full.pkl')

In [22]:
with open(full_model_file, 'rb') as f:
    gee_full = pickle.load(f)

print(gee_full.summary())

                               GEE Regression Results                              
Dep. Variable:                     Correct   No. Observations:                48000
Model:                                 GEE   No. clusters:                        8
Method:                        Generalized   Min. cluster size:                6000
                      Estimating Equations   Max. cluster size:                6000
Family:                           Binomial   Mean cluster size:              6000.0
Dependence structure:         Exchangeable   Num. iterations:                    56
Date:                     Fri, 15 Mar 2024   Scale:                           1.000
Covariance type:                    robust   Time:                         15:53:26
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                           1.6459  

In [18]:
print_wald_chi_square(gee_full)

Wald Chi-Square:
+---------------------------------+--------------------+------------------------+
|             Variable            |     Chi-Square     |        p-value         |
+---------------------------------+--------------------+------------------------+
|            Intercept            | 10.008055874170422 | 0.0015585696172313117  |
|      ContrastHeterogeneity      | 49.15577499337458  | 2.364213012996858e-12  |
|          GridCoarseness         | 58.308691973120965 | 2.2405200226713208e-14 |
|            SessionID            | 6.461520831356995  |  0.011023525757624731  |
| SessionID:ContrastHeterogeneity | 51.35228258690842  | 7.719219789482875e-13  |
|     SessionID:GridCoarseness    | 0.8266047296477383 |   0.3632561477783306   |
+---------------------------------+--------------------+------------------------+


In [40]:
files = glob.glob(os.path.join(results_dir, 'gee_*.pkl'))
files.remove(full_model_file)


for file in sorted(files):
    with open(file, 'rb') as f:
        model = pickle.load(f)
    
    variable = ' '.join(file.split('_')[1:3])
    session = file.split('_')[3].split('.')[0]
    print(f'Session: {session}')
    print(f'Variable: {variable}')
    print(model.summary())
    print_wald_chi_square(model)
    print('\n\n')
    

Session: 1
Variable: contrast heterogeneity
                               GEE Regression Results                              
Dep. Variable:                     Correct   No. Observations:                 6000
Model:                                 GEE   No. clusters:                        8
Method:                        Generalized   Min. cluster size:                 750
                      Estimating Equations   Max. cluster size:                 750
Family:                           Binomial   Mean cluster size:               750.0
Dependence structure:         Exchangeable   Num. iterations:                     6
Date:                     Fri, 15 Mar 2024   Scale:                           1.000
Covariance type:                    robust   Time:                         16:06:12
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept           

In [45]:
from prettytable import PrettyTable

table = PrettyTable()

table.field_names = ['session', 'predictor', 'beta-coefficient', 'confidence interval', 'sdt error', 'Wald chi-square', 'p-value']

for file in sorted(files):
    with open(file, 'rb') as f:
        results = pickle.load(f)
    
    session = file.split('_')[3].split('.')[0]
    for predictor in results.model.exog_names:
        if predictor == 'Intercept':
            continue
        table.add_row([session, predictor, results.params[1], results.conf_int()[1][0], results.bse[1], results.wald_test(predictor, scalar=True).statistic, results.pvalues[1]])

/tmp/ipykernel_150379/1075944853.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  table.add_row([session, predictor, results.params[1], results.conf_int()[1][0], results.bse[1], results.wald_test(predictor, scalar=True).statistic, results.pvalues[1]])
/tmp/ipykernel_150379/1075944853.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  table.add_row([session, predictor, results.params[1], results.conf_int()[1][0], results.bse[1], results.wald_test(predictor, scalar=True).statistic, results.pvalues[1]])
/tmp/ipykernel_150379/1075944853.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecat

In [52]:
results.bse

Intercept                0.305826
ContrastHeterogeneity    0.345900
dtype: float64